# Monthly wages for females and males
In LOCAL CURRENCY UNITS

In [1]:
import pandas as pd
import numpy as np
import os
import glob
import seaborn as sns
import matplotlib.pyplot as plt
import json
import csv
from collections import defaultdict
import functools
import operator

In [2]:
# Read csv files from No Ceilings project folder
path = "/Users/ericaxia/Downloads/Github/project-girlboss/data/noceilings-data-master/csv"
extension = 'csv'
os.chdir(path)
result = glob.glob('*.{}'.format(extension))
# Narrow down to just the files I want
files = ['MONWAGFE.csv', 'MONWAGMA.csv']
dfs = []
for f in files:
    df = pd.read_csv(f)
    dfs.append((f, df))    

## Let's narrow down to a chosen year

In [3]:
os.chdir("/Users/ericaxia/Downloads/Github/dsci554/554_Project_Code/Project/d3layout_data")

In [4]:
## NEW: map ISO -> Country name for better understandability in graph
with open('iso_to_country_names.csv', mode='r') as infile:
    reader = csv.reader(infile)
    country_names = {rows[0]: rows[1] for rows in reader}
    # print(country_names)

country_names_dict = { v: k for k, v in country_names.items()}
# country_names_dict


In [5]:
def convert_iso_to_name(iso):
    # print(x)
    if iso in country_names_dict:
        return country_names_dict[iso]
    else:
        return iso

In [6]:
wages_fem = dfs[0][1]  # monthly wages for females
wages_m = dfs[1][1] # monthly wages for males


In [7]:
wages_fem = wages_fem.iloc[0:85, :]  # rm bottom descrip rows
wages_m = wages_m.iloc[0:85, :]  # rm bottom descrip rows

In [8]:
# wages_fem.head()

In [9]:
## Missing values by year
# wages_fem.isna().sum().sort_values()[0:5]

In [10]:
# narrow down to countries that DO have data for 2011
wages_fem = wages_fem[~wages_fem['2011'].isna()]
wages_m = wages_m[~wages_m['2011'].isna()]
print(wages_fem.shape, wages_m.shape) # (26,18) -> we have 26 countries

(26, 18) (26, 18)


In [17]:
# TODO: clean dfs, and export to JSON format needed for bar charts
wages_fem = wages_fem.fillna(axis=0, method='pad')
wages_m = wages_m.fillna(axis=0, method='pad')

In [23]:
wages_fem['gender'] = 'female'
wages_m['gender'] = 'male'


In [24]:
wages_fem.isna().sum().sum()
wages_m.isna().sum().sum()

0

In [25]:
c = 'AUS'
f1 = wages_fem[wages_fem['ISO'] == c]
m1 = wages_m[wages_m['ISO'] == c]
test = pd.concat([f1, m1], axis=0)
test.head()

,ISO,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,gender
2,AUS,1867.23,1923.89,1996.15,2043.60,2081.95,2187.25,2286.27,2347.15,2450.61,2552.77,2680.17,2790.67,2931.82,3047.20,3181.43,3341.43,3473.27,female
2,AUS,2846.68,2924.68,3013.51,3116.64,3172.43,3299.18,3439.15,3598.51,3790.26,3893.07,4102.37,4289.13,4502.88,4689.64,4903.38,5152.77,5373.88,male


In [ ]:
# old code below:

In [ ]:
def clean_dfs(dfs, gender, year):
    countries = []
    json_names = []
    for i in range(len(dfs)):
        name = dfs[i][0].strip(".csv")
        df = dfs[i][1]
        df = df[['ISO', year]]
        df.dropna(inplace=True)  # drop nans
        df.columns = ['name', 'value']  # rename cols
        # change ISO to full country
        df['name'] = df['name'].apply(lambda x: convert_iso_to_name(x))
        df['gender'] = gender  # set gender as new column
        # drop weird ones that dont match any full country name
        df = df[~df["name"].str.isupper()]
        df.to_json(f"{name}.json", orient='records')  # export to json
        # print(f"Exported {name} data with shape {df.shape}")
        json_names.append(f"{name}.json")
        # # get unique countries
        names = list(df['name'].unique())
        countries.append(names)
    return json_names, countries


In [ ]:
# year = '2012'  # chosen year for plot
# json_names_fem, countries_f = clean_dfs(dfs_female, 'female', year)
# json_names_male, countries_m = clean_dfs(dfs_male, 'male', year)

In [ ]:
## continue where i left off

In [ ]:
# Read back in the json files saved
os.chdir("/Users/ericaxia/Downloads/Github/dsci554/554_Project_Code/Project/d3layout_data")
extension = 'json'
result = glob.glob('*.{}'.format(extension))
# Narrow down to just the files I want
f_result = [f for f in result if f in json_names_fem]
m_result = [f for f in result if f in json_names_male]

In [ ]:
# import csv file with full dataset name mappings
with open('data_names_male.csv', mode='r') as infile:
    reader = csv.reader(infile)
    dataset_names_male = {rows[0]: rows[1].rstrip(", Male") for rows in reader}
    print(dataset_names_male)

In [ ]:
# import csv file with full dataset name mappings
with open('data_names.csv', mode='r') as infile:
    reader = csv.reader(infile)
    dataset_names_female = {rows[0]: rows[1].rstrip(", Female") for rows in reader}
    print(dataset_names_female)

# Create big json file

In [ ]:
data_dict = defaultdict(list)
# female
for f in f_result:
    ds_code = f.split('.json')[0]
    ds_name = dataset_names_female[ds_code]
    # print(ds_name)
    with open(f, "r") as read_file:
        data = json.load(read_file)  # list of dicts
        data_dict[ds_name].append(data)
# print(len(data_dict))

In [ ]:
# male
for f in m_result:
    ds_code = f.split('.json')[0]
    ds_name = dataset_names_male[ds_code]
    print(ds_name)
    with open(f, "r") as read_file:
        data = json.load(read_file)  # list of dicts
        data_dict[ds_name].append(data)


In [ ]:
# for occupation, list of lists ([male list], [female list]) 
## flatten the list of lists
data_dict2 = defaultdict(list)
for occ, lol in data_dict.items():
    print(occ)
    if len(lol) == 2:
        data_dict2[occ] = lol[0] + lol[1]
    elif len(lol) == 1:
        data_dict2[occ] = lol

In [ ]:
export_file = { "name": "employment", "children": [] }

for occ, l in data_dict2.items():
    inner_dict = { "name": occ, "children": l }
    export_file["children"].append(inner_dict)


In [ ]:
print(len(export_file['children']))
print(type(export_file['children']))

In [ ]:
with open("all_employment_data.json", "w") as write_file:
    json.dump(export_file, write_file)